In [10]:
%load_ext autoreload
%autoreload 2
import os
import sys
import time
import math
import numpy as np
import sklearn
from utils import *
from models.resnetF import *
from plot_cnn import *
%matplotlib inline
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm, tqdm_notebook
import torchvision.models as models
import torch, torchvision

import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.datasets as datasets
import torch.utils.data as data
import torchvision.transforms as transforms
from torch.autograd import Variable
from shap_utils import *

from torch.optim import lr_scheduler
import copy
#hyper parameters
batch_size = 100
epochs = 30
# train_size = 1000
# test_size = 1000

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms

data_transforms = { 'train': transforms.Compose([transforms.ToTensor()]),
                    'val'  : transforms.Compose([transforms.ToTensor(),]) }
data_dir = "data/tiny-imagenet-200"
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=64)
              for x in ['train', 'val']}
print(len(image_datasets['train']))
middle_dataset, pre_dataset = torch.utils.data.random_split(image_datasets['train'], [5000, 95000])
train_dataset, cal_dataset = torch.utils.data.random_split(middle_dataset, [2500, 2500])

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=64)
test_dataloader = torch.utils.data.DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=True, num_workers=64)
cal_dataloader = torch.utils.data.DataLoader(cal_dataset, batch_size=batch_size, shuffle=True, num_workers=64)
pre_dataloader = torch.utils.data.DataLoader(pre_dataset, batch_size=batch_size, shuffle=True, num_workers=64)

# pre_dataloader = dataloaders['val']
cal_size = len(cal_dataset)

train_size = len(train_dataset)
test_size = len(image_datasets['val'])
pre_size = len(pre_dataset)
print(train_size, cal_size, test_size, pre_size)

100000
2500 2500 10000 95000


In [16]:
def tiny_train(phase, model, device, dataloaders, dataset_sizes, optimizer, criterion, scheduler, epochs=1):   
    for epoch in range(epochs):
        print('Epoch {}/{}'.format(epoch+1, epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        if phase == 'train':
            scheduler.step()
            model.train()  # Set model to training mode
        else:
            model.eval()   # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for i,(inputs, labels) in enumerate(dataloaders):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            with torch.set_grad_enabled(phase == 'train'):
                *_, outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        print("\repochs: {}/{}, Loss: {}.".format(epoch+1, epochs, loss.item() * inputs.size(0)), end="")
  
        epoch_loss = running_loss / dataset_sizes
        epoch_acc = running_corrects.double() / dataset_sizes
        if phase == 'train':
            avg_loss = epoch_loss
            t_acc = epoch_acc
        elif phase == 'val':
            val_loss = epoch_loss
            val_acc = epoch_acc    
    if phase == 'train':
        print('Train Loss: {:.4f} Acc: {:.4f}'.format(avg_loss, t_acc))
    elif phase == 'val':
        print('Val Loss: {:.4f} Acc: {:.4f}'.format(val_loss, val_acc))
        return val_acc
    return

In [65]:
seed = 0
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic=True
from models.resnetF import *
device = torch.device('cuda')
model = resnet18(pretrained=True)
model.avgpool = nn.AdaptiveAvgPool2d(1)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 200)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()
scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

model = model.to(device)
epochs = 30
tiny_train("train", model, device, train_dataloader, train_size, optimizer, criterion, scheduler, epochs)
tiny_train("val", model, device, test_dataloader, test_size, optimizer, criterion, scheduler)

Epoch 1/30
----------
epochs: 1/30, Loss: 542.0825004577637.Epoch 2/30
----------
epochs: 2/30, Loss: 491.0949230194092.Epoch 3/30
----------
epochs: 3/30, Loss: 443.6898708343506.Epoch 4/30
----------
epochs: 4/30, Loss: 381.7734479904175.Epoch 5/30
----------
epochs: 5/30, Loss: 350.2703905105591.Epoch 6/30
----------
epochs: 6/30, Loss: 362.526273727417.Epoch 7/30
----------
epochs: 7/30, Loss: 350.4526138305664.Epoch 8/30
----------
epochs: 8/30, Loss: 338.5174036026001.Epoch 9/30
----------
epochs: 9/30, Loss: 334.5339059829712.Epoch 10/30
----------
epochs: 10/30, Loss: 336.1944913864136.Epoch 11/30
----------
epochs: 11/30, Loss: 334.90304946899414.Epoch 12/30
----------
epochs: 12/30, Loss: 343.7199354171753.Epoch 13/30
----------
epochs: 13/30, Loss: 336.76133155822754.Epoch 14/30
----------
epochs: 14/30, Loss: 342.15545654296875.Epoch 15/30
----------
epochs: 15/30, Loss: 342.21439361572266.Epoch 16/30
----------
epochs: 16/30, Loss: 342.50268936157227.Epoch 17/30
----------

tensor(0.0632, device='cuda:0', dtype=torch.float64)

In [66]:
torch.multiprocessing.set_sharing_strategy('file_system')
deep_f_train = []
y_train = []
x_train = []
deep_f_cal = []
y_cal = []
x_cal = []
for i,(X, y) in enumerate(train_dataloader):
    X = X.to(device).float()
    out_fc1, y_pre = model(X)
    deep_f_train.append(out_fc1.view(out_fc1.size(0), -1).cpu().detach().numpy())
    x_train.append(X.view(X.size(0), -1).cpu().detach().numpy())
    y_train.append(y.numpy())

deep_f_train = np.concatenate(deep_f_train) # deep features are not normalized
y_train = np.concatenate(y_train)
x_train = np.concatenate(x_train)
print(x_train.shape, y_train.shape, deep_f_train.shape)

for i,(X, y) in enumerate(cal_dataloader):
#     print(i)
    X = X.to(device).float()
    out_fc1, y_pre = model(X)
    deep_f_cal.append(out_fc1.view(out_fc1.size(0), -1).cpu().detach().numpy())
    x_cal.append(X.view(X.size(0), -1).cpu().detach().numpy())
    y_cal.append(y.numpy())
deep_f_cal = np.concatenate(deep_f_cal) # deep features are not normalized
y_cal = np.concatenate(y_cal)
x_cal = np.concatenate(x_cal)
print(x_cal.shape, y_cal.shape, deep_f_cal.shape)

(2500, 12288) (2500,) (2500, 512)
(2500, 12288) (2500,) (2500, 512)


In [114]:
kmin = 1
kmax = 10
kinterval = 1
fc1_knn_values = [[] for _ in range(math.ceil((kmax-kmin)/kinterval))] # deep features
loo_fc1_knn_values = [[] for _ in range(math.ceil((kmax-kmin)/kinterval))] # deep features

fc1_scores = [[] for _ in range(math.ceil((kmax-kmin)/kinterval))]

for i, k in enumerate(range(kmin, kmax, kinterval)):
    print("neighbour number:", k)
    loo_fc1_knn_values[i],fc1_scores[i], fc1_false = loo_knn_shapley(k, deep_f_train, deep_f_cal, y_train, y_cal)
    fc1_knn_values[i],*_ = old_knn_shapley(k, deep_f_train, deep_f_cal, y_train, y_cal)


neighbour number: 1


neighbour number: 6


In [115]:
print(fc1_knn_values[0][:10])
x_heldout = []
y_heldout = []
for i,(X, y) in enumerate(test_dataloader):
    x_heldout.append(X.view(X.size(0), -1).cpu().detach().numpy())
    y_heldout.append(y.numpy())
y_heldout = np.concatenate(y_heldout)
x_heldout = np.concatenate(x_heldout)  
print(y_heldout.shape)

x_pre = []
y_pre = []
for i,(X, y) in enumerate(pre_dataloader):
    x_pre.append(X.view(X.size(0), -1).cpu().detach().numpy())
    y_pre.append(y.numpy())
x_pre = np.concatenate(x_pre)
y_pre = np.concatenate(y_pre)  
print(y_pre.shape)

[ 8.07750307e-06 -1.36472570e-05  2.08875064e-05  2.71985499e-05
  2.33321028e-06  5.17637251e-04 -8.14243583e-07  3.73426756e-05
  1.89738024e-04  4.88869666e-04]
(10000,)
(95000,)


In [116]:
loo_pre_score = []
tmc_pre_score = []
knn_pre_score = [[] for _ in range(math.ceil((kmax-kmin)/kinterval))]
loo_knn_pre_score = [[] for _ in range(math.ceil((kmax-kmin)/kinterval))]
knn_pre_idx = [[] for _ in range(math.ceil((kmax-kmin)/kinterval))]
loo_knn_pre_idx = [[] for _ in range(math.ceil((kmax-kmin)/kinterval))]

print(fc1_knn_values[0][:10])

for i in range(len(fc1_knn_values)):
    fc1_knn_values[i] = fc1_knn_values[i] / np.linalg.norm(fc1_knn_values[i])
    loo_fc1_knn_values[i] = loo_fc1_knn_values[i] / np.linalg.norm(loo_fc1_knn_values[i])
print(fc1_knn_values[0][:10])


print(loo_fc1_knn_values[0][:10])

print(loo_fc1_knn_values[0][-10:])


knn_values_idx = [[] for _ in range(math.ceil((kmax-kmin)/kinterval))]
loo_knn_values_idx = [[] for _ in range(math.ceil((kmax-kmin)/kinterval))]

for i in range(len(fc1_knn_values)):
    knn_values_idx[i] = np.where(fc1_knn_values[i] > -100.0000)[0]
    loo_knn_values_idx[i] = np.where(loo_fc1_knn_values[i] > -100.0000)[0]

    print("knn_values:", len(knn_values_idx[i]))
    print("loo_knn_values:", len(loo_knn_values_idx[i]))

clf_knn = [[] for _ in range(math.ceil((kmax-kmin)/kinterval))]
clf_loo_knn = [[] for _ in range(math.ceil((kmax-kmin)/kinterval))]

for i in range(len(fc1_knn_values)):
#     clf_knn[i] = return_model('mlpreg')
    clf_knn[i] = return_model('RandomForestReg')

    clf_knn[i].fit(x_train[knn_values_idx[i]], fc1_knn_values[i][knn_values_idx[i]])
    print("finish fit")

for i in range(len(loo_fc1_knn_values)):
#     clf_loo_knn[i] = return_model('RandomForestReg')
    clf_loo_knn[i] = return_model('mlpreg')
    clf_loo_knn[i].fit(x_train[loo_knn_values_idx[i]], loo_knn_values_idx[i][loo_knn_values_idx[i]])
    print("finish fit")

[ 8.07750307e-06 -1.36472570e-05  2.08875064e-05  2.71985499e-05
  2.33321028e-06  5.17637251e-04 -8.14243583e-07  3.73426756e-05
  1.89738024e-04  4.88869666e-04]
[ 8.84888276e-04 -1.49505331e-03  2.28822067e-03  2.97959378e-03
  2.55602555e-04  5.67070208e-02 -8.92001643e-05  4.09088001e-03
  2.07857492e-02  5.35555396e-02]
[0.         0.         0.         0.         0.         0.05479966
 0.         0.         0.         0.05479966]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
knn_values: 2500
loo_knn_values: 2500
knn_values: 2500
loo_knn_values: 2500
finish fit
finish fit
finish fit
finish fit


In [117]:
for i in range(len(knn_values_idx)):
    knn_pre_score[i] = clf_knn[i].predict(x_pre)
    knn_pre_idx[i]=np.argsort(knn_pre_score[i])

for i in range(len(loo_fc1_knn_values)):
    loo_knn_pre_score[i] = clf_loo_knn[i].predict(x_pre)
    loo_knn_pre_idx[i]=np.argsort(loo_knn_pre_score[i])
    
print((knn_pre_score[0][:10]))
print((loo_knn_pre_score[0][:10]))
print(knn_pre_idx[0][-100:])

[ 0.00020123  0.00614738  0.00098427  0.01952971  0.00651056  0.01658965
  0.00782489  0.00945298  0.00250302 -0.0022188 ]
[ 756.44348538  442.79842198 1258.99345873  513.39372426 1485.12419292
  273.11735032  941.82725226  562.57933618  592.16990526 1016.71952644]
[11012 88915 67643 50583 74800  7092 71058 20305 10697 89971 83910 57845
 92481  2724    19 83934 47234  5729 57580 22259 35155 94957 30791 11924
 13643 65170 56372 33673 34971 68403 47570 74267 36238 60901 62692 23296
 14512 51752 38338 11235 57898  8342  5297  5668 81950 93005  2853 24983
  1922 87564 50153 39214 30588  2846 22086 47652 54387 18881 86117  5153
 24385  9410 78515 27356 84880 81084 86378  2768 53834 77068 79023 60339
 16859 39265 36329  7589 92802 50437 60759 57976 80103 87139 25904 55391
 75316 67717 19463 86417 16282 24549 87351 49591 31329 75232 66612 87513
 77865 69121 36916 19369]


In [118]:
import pickle
with open("exp_data/DA_tinyimagenet/inf_fun.pkl", "rb") as f:
    d = pickle.load(f)
    inf_values = d["loo"]
    
print(inf_values[:10])
inf_values = inf_values / np.linalg.norm(inf_values)
print(inf_values[:10])


inf_pre_score = []
inf_values_idx = np.where(inf_values > -100.0000)[0]
clf_inf = return_model('RandomForestReg')

clf_inf.fit(x_train[inf_values_idx], inf_values[inf_values_idx])
print("finish fit")

[0.17172956466674805, 0.26072579622268677, 0.1966152191162109, 0.16916394233703613, 0.12126377224922182, 0.021961569786071777, 0.27344438433647156, -0.38717180490493774, -0.759654700756073, -0.15183240175247192]
[ 0.00609461  0.00925305  0.00697779  0.00600356  0.0043036   0.00077941
  0.00970443 -0.01374056 -0.02695982 -0.00538847]
finish fit


In [119]:
inf_pre_score = clf_inf.predict(x_pre)
print(inf_pre_score[:10])
inf_pre_idx = np.argsort(inf_pre_score)
print(inf_values)

[-0.00023628  0.00318097  0.0027922  -0.0069073  -0.00024983  0.00185853
  0.00483222 -0.00094235  0.0038929   0.0034466 ]
[ 0.00609461  0.00925305  0.00697779 ... -0.01901985  0.0040502
  0.0087479 ]


In [1]:
# print(avg_knn_values)
from plot_resnet import *
seed = 48
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic=True

sx_train = torch.from_numpy(x_train).contiguous().view(-1, 3,64,64)
sy_train = torch.from_numpy(y_train).view(-1,).long()
print("train_size:", sx_train.shape)
sx_test = torch.from_numpy(x_heldout).contiguous().view(-1, 3,64,64)
sy_test = torch.from_numpy(y_heldout).view(-1,).long()
print("test_size:", sx_test.shape)
sx_pre = torch.from_numpy(x_pre).contiguous().view(-1, 3,64,64)
sy_pre = torch.from_numpy(y_pre).view(-1,).long()
print("test_size:", sx_pre.shape)

plot_resnet_acq(knn_pre_idx, loo_knn_pre_idx, [inf_pre_idx], kmin, kmax, kinterval, sx_train, sy_train, sx_test, sy_test, sx_pre, sy_pre, batch_size, 
            epochs=10, HtoL=True)